In [1]:
import torch
import torch.nn as nn
import pandas as pd
import scipy.sparse as sp
import numpy as np
from tqdm import  tqdm
import torch.optim as optim

### Verify Pytorch's version

In [2]:
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2.3.0a0+6ddf5cf85e.nv24.04


### Take a quick look of the training dataset

In [3]:
data = pd.read_table("Data/ml-1m.train.rating",sep="\t",header=None)
data.head(5)

,0,1,2,3
0,0,32,4,978824330
1,0,34,4,978824330
2,0,4,5,978824291
3,0,35,4,978824291
4,0,30,4,978824291


## Define NCF

In [4]:
class NCF(object):
    def __init__(self, config):
        self.config = config
        self._num_users = config['num_users']
        self._num_items = config['num_items']
        self._X = config['layer_X']
        self._factor = config['factor']
        self._embedding_size_gmf = self._factor
        self._embedding_size_mlp = self._factor*(2**(self._X-1))

        self._embedding__user_gmf = nn.Embedding(self._num_users, self._embedding_size_gmf)
        self._embedding__item_gmf = nn.Embedding(self._num_items, self._embedding_size_gmf)

        if self._X > 0:
            self._embedding__user_mlp = nn.Embedding(self._num_users, self._embedding_size_mlp)
            self._embedding__item_mlp = nn.Embedding(self._num_items, self._embedding_size_mlp)

            self._fc_layers = nn.ModuleList()
            for idx in range(self._X-1, -1, -1):
                in_size = self._factor*(2**(idx+1))
                out_size = self._factor*(2**idx)
                self._fc_layers.append(nn.Linear(in_size, out_size))
        self._out_fc = nn.Linear(self._factor, 1, bias=False)
        
        self._activate1 = nn.Sigmoid()
        self._activate2 = nn.ReLU()
    def __repr__(self):
        return ""

### GMF

In [5]:
class GMF(NCF,nn.Module):
    def __init__(self, config):
        nn.Module.__init__(self)
        NCF.__init__(self, config)

    def forward(self, user_idx, item_idx):
        user_embedding = self._embedding__user_gmf(user_idx)
        item_embedding = self._embedding__item_gmf(item_idx)
        pointwise_vector = torch.mul(user_embedding, item_embedding)
        logit = self._out_fc(pointwise_vector)
        prob = self._activate1(logit)
        return prob

### MLP

In [6]:
class MLP(NCF,nn.Module):
    def __init__(self, config):
        nn.Module.__init__(self)
        NCF.__init__(self, config)

    def forward(self, user_idx, item_idx):
        user_embedding = self._embedding__user_mlp(user_idx)
        item_embedding = self._embedding__item_mlp(item_idx)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)
        for _, layer in enumerate(self._fc_layers):
            vector = layer(vector)
            vector = self._activate2(vector)
        logit = self._out_fc(vector)
        prob = self._activate1(logit)
        return prob

### NeuMF

In [7]:
class NeuMF(NCF,nn.Module):
    def __init__(self, config):
        nn.Module.__init__(self)
        NCF.__init__(self, config)
        self._neumf_fc = nn.Linear(self._factor*2, 1, bias=False)
    
    def forward(self, user_idx, item_idx):
        user_embedding_gmf = self._embedding__user_gmf(user_idx)
        item_embedding_gmf = self._embedding__item_gmf(item_idx)
        pointwise_vector_gmf = torch.mul(user_embedding_gmf, item_embedding_gmf)

        user_embedding_mlp = self._embedding__user_mlp(user_idx)
        item_embedding_mlp = self._embedding__item_mlp(item_idx)
        vector_mlp = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)
        for _, layer in enumerate(self._fc_layers):
            vector_mlp = layer(vector_mlp)
            vector_mlp = self._activate2(vector_mlp)

        vector_neumf = torch.cat([pointwise_vector_gmf, vector_mlp], dim=-1)
        logit = self._neumf_fc(vector_neumf)
        prob = self._activate1(logit)
        return prob

In [8]:
GMF_config = {'num_users': 6040, 'num_items': 3706, 'factor': 8, 'layer_X': 0}
MLP_config = {'num_users': 6040, 'num_items': 3706, 'factor': 8, 'layer_X': 3}
NeuMF_config = {'num_users': 6040, 'num_items': 3706, 'factor': 8, 'layer_X': 3}

In [9]:
GMF_model = GMF(GMF_config)
GMF_model.to(device)

In [10]:
# generate random user_idx and item_idx
user_idx = torch.randint(0, GMF_config['num_users'], (1,)).to(device)
item_idx = torch.randint(0, GMF_config['num_items'], (1,)).to(device)

# using torch.jit.trace to trace model
traced_model = torch.jit.trace(GMF_model, (user_idx, item_idx))

print(traced_model.graph)

graph(%self.1 : __torch__.GMF,
      %user_idx : Long(1, strides=[1], requires_grad=0, device=cuda:0),
      %item_idx : Long(1, strides=[1], requires_grad=0, device=cuda:0)):
  %_activate1 : __torch__.torch.nn.modules.activation.Sigmoid = prim::GetAttr[name="_activate1"](%self.1)
  %_out_fc : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="_out_fc"](%self.1)
  %_embedding__item_gmf : __torch__.torch.nn.modules.sparse.___torch_mangle_0.Embedding = prim::GetAttr[name="_embedding__item_gmf"](%self.1)
  %_embedding__user_gmf : __torch__.torch.nn.modules.sparse.Embedding = prim::GetAttr[name="_embedding__user_gmf"](%self.1)
  %50 : Tensor = prim::CallMethod[name="forward"](%_embedding__user_gmf, %user_idx)
  %51 : Tensor = prim::CallMethod[name="forward"](%_embedding__item_gmf, %item_idx)
  %input.1 : Float(1, 8, strides=[8, 1], requires_grad=1, device=cuda:0) = aten::mul(%50, %51) # /tmp/ipykernel_35005/167587794.py:9:0
  %52 : Tensor = prim::CallMethod[name="forward"](%_ou

In [11]:
MLP_model = MLP(MLP_config)
MLP_model.to(device)

In [12]:
NeuMF_model = NeuMF(NeuMF_config)
NeuMF_model.to(device)

## Loading dataset

In [13]:
class Dataset(object):
    def __init__(self, path):
        self.trainMatrix = self.load_rating_file_as_matrix(path + ".train.rating")
        self.testRatings = self.load_rating_file_as_list(path + ".test.rating")
        self.testNegatives = self.load_negative_file(path + ".test.negative")
        assert len(self.testRatings) == len(self.testNegatives)
        self.num_users, self.num_items = self.trainMatrix.shape
        
    def load_rating_file_as_list(self, filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList
    
    def load_negative_file(self, filename):
        negativeList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                negatives = []
                for x in arr[1: ]:
                    negatives.append(int(x))
                negativeList.append(negatives)
                line = f.readline()
        return negativeList
    
    def load_rating_file_as_matrix(self, filename):
        num_users, num_items = 0, 0
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                u, i = int(arr[0]), int(arr[1])
                num_users = max(num_users, u)
                num_items = max(num_items, i)
                line = f.readline()
        mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
                if (rating > 0):
                    mat[user, item] = 1.0
                line = f.readline()    
        return mat

In [14]:
dataset = Dataset("./Data/"+"ml-1m")
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape

### Adding negative samples to trainset

In [15]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

### Create traindataloader

In [16]:
from torch.utils.data import Dataset, DataLoader
class mlDataset(Dataset):
    def __init__(self, user_input, item_input, labels):
        self.user_input = user_input
        self.item_input = item_input
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, index):
        return self.user_input[index], self.item_input[index], self.labels[index]
user_input, item_input, labels = get_train_instances(train, num_negatives=4)
train_dataset = mlDataset(user_input, item_input, labels)
train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=True)

## Evaluation Method

In [17]:
def evaluate(model,topk):
    class testDataset(Dataset):
        def __init__(self, rating, negative_lists):
            self.rating = rating
            self.negative_lists = negative_lists
        def __len__(self):
            return len(self.rating)
        def __getitem__(self, index):
            return self.rating[index], self.negative_lists[index]

    def HR_NDCG(testloader):
        model.eval()
        ht = 0; ndcg = 0
        with torch.no_grad():
            for rating, negatives in testloader:
                user_idxs = torch.tensor([x[0] for x in rating], dtype=torch.long).to(device)
                pos_item_idxs = torch.tensor([x[1] for x in rating], dtype=torch.long).to(device)
                neg_item_idxs = torch.stack([x.clone().detach() for x in negatives]).to(device)
                print(pos_item_idxs.size())
                print(neg_item_idxs.size())

                pos_scores = model(user_idxs, pos_item_idxs).unsqueeze(1)  # (batch_size, 1)
                neg_scores = model(user_idxs.unsqueeze(1).repeat(1, neg_item_idxs.size(1)), neg_item_idxs)  # (batch_size, num_negatives)
                all_scores = torch.cat((pos_scores, neg_scores), dim=1)  # (batch_size, num_negatives+1)

                # calculate HR
                _, topk_indices = torch.topk(all_scores, topk, dim=1, largest=True, sorted=True)
                ht += torch.sum((topk_indices == 0).int()).item()  # 0 is the index of positive example in concatenated scores

                # calculate NDCG
                sorted_scores, _ = torch.sort(all_scores, dim=1, descending=True)
                _, rankings = torch.where(pos_scores == sorted_scores)
                ndcg += torch.sum(1 / torch.log2(rankings + 2)).item()

        return ht / len(testloader.dataset), ndcg / len(testloader.dataset)

    test_dataset = testDataset(testRatings, testNegatives)
    test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=False)
    hr, ndcg = HR_NDCG(test_loader)
    return hr, ndcg

## Training Model

In [18]:
topk = 10

### GMF

In [19]:
optimizer = optim.Adam(GMF_model.parameters(), lr=0.001)
# optimizer = optim.SGD(GMF_model.parameters(), lr=0.001)
criterion = nn.BCELoss() #期望的输入是经过sigmoid函数处理的，此处应该选用BCELoss而不是BCEWithLogitsLoss

In [20]:
num_epochs = 30
for epoch in tqdm(range(num_epochs)):
    GMF_model.train()
    running_loss = 0
    for user_idxs, item_idxs, labels in train_loader:
        optimizer.zero_grad()

        user_idxs = user_idxs.to(device)
        item_idxs = item_idxs.to(device)
        labels = labels.to(device).to(torch.float32)

        outputs = GMF_model(user_idxs, item_idxs)
        loss = criterion(outputs, labels.unsqueeze(1))
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
    HR, NDCG = evaluate(GMF_model, topk)
    print(f'HR@{topk}: {HR}, NDCG@{topk}: {NDCG}')

  0%|          | 0/30 [00:41<?, ?it/s]

Epoch 1/30, Loss: 0.6930796504020691
torch.Size([2])
torch.Size([99, 256])


RuntimeError: The size of tensor a (2) must match the size of tensor b (99) at non-singleton dimension 0